In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'Length'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

strategy_version_input=751
attribute_id_input=2425

In [3]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# buckets

In [5]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r', encoding='utf-8') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
    
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    
#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))

from fractions import Fraction
def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))
# measure='This is a fraction: 100 1/64'
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A



In [6]:
# df_fix=pd.read_csv('BBBY - length-__-matches.csv')
# df_fix[curation_col]=df_fix[curation_col].apply(lambda x: re.sub(r'inength','in',str(x)))
# df_fix[curation_col].explode().value_counts()

In [7]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df = df[df['bucket_id'].isin(buckets)]
len(df)

501

In [8]:
# print(len(df[df['matches_length'].astype(str)=='[]']))
# df[df['matches_length'].astype(str)=='[]']

In [9]:
del df['product_id']
del df['customer_id']
del df['attributes']
del df['bucket_id']
del df['values']

# NA

In [10]:
regex_pattern_length=r'(?i)((?<!cord.measures.)\d+\.\d+\W{0,3}(?:(?:L\b)|(?:Length))\b)|((?<!cord.measures.)\d+.?\d+\/\d+\W{0,3}(?:(?:L\b)|(?:Length))\b)|((?<!cord.measures.)(?<!\.)(?<!\/)(?<!\d)\d+(?!\.\d)\W{0,3}(?:(?:L\b)|(?:Length))\b)|((?<!cord.measures.)(?<!\.)(?<!\/)(?<!\d)\d+(?!\.)\W{0,3}(?:(?:L\b)|(?:Length))\b)|()'         
df['matches_length'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
regex_pattern_height=r'''(?i)((?<!\.)(?<!up.to.)(?<!\w)(?<!\/)(?<!\/)(?<!\.)\d+(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?\\?(?:"?|”?|'{0,2}|\Winc?h?|\Wft|\Wfoot)\W(?:H(?:eight)?|square\W?inch|square\W?foot)\b(?!.bulb)(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)(?!.?options))|()'''
# (?i)(\d+\.\d+[\W]?[\W]?[\W]?He?i?g?h?t?\b)|(\d+.?\d+\/\d+[\W]?[\W]?[\W]?He?i?g?h?t?\b(?!options))|((?<!\.)(?<!up.to.)(?<![\w])(?<!\/)(?<!\/)\d+(?!\.\d)(?<!\d\\?\"?\-\d\d)[\W]?[\W]?[\W]?He?i?g?h?t?(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)(?!.?options)\b)|()'        
df['matches_height'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
regex_pattern_width=r'(?i)(\d+\.\d+.?.?.?Wi?d?t?h?\b)|(\d+.?\d+\/\d+.?.?.?Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Wi?d?t?h?\b)|()'
df['matches_width'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))
regex_pattern_diameter=r'(?i)(\d+\.\d+.?.?.?Diam?e?t?e?r?\b)|(\d+.?\d+\/\d+.?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Diam?e?t?e?r?\b)|()'
df['matches_diameter'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
regex_pattern_ft=r"\d'.?.?.?[LWHD]|\d\'.?diameter|(\bround\b)|(circ)|(lamp)"
df['matches_ft'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
regex_pattern_blank=r'\d'
df['new_matches'] = df['matches_diameter'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
regex_pattern_blank=r'(?i)(\d+.?\d*\/?\d*.?I?n?c?h?"?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?)|(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
df['blank_matches'] = df['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df['blank_matches_long'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))    

# Length

In [11]:
heights=df[(df['matches_length'].astype(str)!='[]')]
heights['matches_height']=heights['matches_length'].apply(lambda x: natsorted(x))
## heights['matches_height']=heights['matches_height'].apply(lambda x: fix_text(f'{x}',normalization='NFKC'))
##encoding
# heights['height_final']=heights['matches_height'].apply(lambda x: re.sub(r'''(?i)(\\?\"?\”?(?:inc?h?)?(?<!').(Height|H))|(\\?('')\W?(Height|H))|(\\" H)|(\\ in)|(square\W?Inch)''',' in',str(x))).apply(lambda x: re.sub(r'''(\\?(?:(?<!')'(?!'))?(?:f?t?)?(?:f?o?o?t?)?\W?(Height|H))|(square\W?foot)|(\'\\xa.{0,2}H)''',' ft',str(x))).apply(lambda x: re.sub(r'''(\'.?xa.{0,3}in ft)''',' ft',str(x))).apply(lambda x: re.sub(r'''(?i)(\\ in)''',' in',str(x))).apply(lambda x: re.sub(r'(?i)\"xa0W|xa0W',' in',str(x))).apply(lambda x: re.sub(r'''(?i)\'\\xa in''',' ft',str(x)))
heights['height_final']=heights['matches_height'].apply(lambda x: re.sub(r'''(?i)(\\\"\s?L)|(\\?\"?\”?(?:inc?h?)?(?<!').(Length|L))|(\\?('')\W?(Length|L))|(\\ in)|(square\W?Inch)''',' in',str(x))).apply(lambda x: re.sub(r'''(\\?(?:(?<!')'(?!'))?(?:f?t?)?(?:f?o?o?t?)?\W?(Length|L))|(square\W?foot)|(\'\\xa.{0,2}L)''',' ft',str(x))).apply(lambda x: re.sub(r'''(?i)(\\ in)''',' in',str(x)))#.apply(lambda x: re.sub(r'''(\'.?xa.{0,3}in ft)''',' ft',str(x)))#.apply(lambda x: re.sub(r'(?i)\"xa0W|xa0W',' in',str(x)))#.apply(lambda x: re.sub(r'''(?i)\'\\xa in''',' ft',str(x)))
heights['height_final']=heights['height_final'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
heights['height_final']=heights['height_final'].apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'" ','"',str(x))).apply(lambda x: re.sub(r'""','"',str(x))).apply(lambda x: re.sub(r'''(?i)(\\")''','"',str(x)))
heights['height_final']=heights['height_final'].apply(lambda x: re.sub(r'''(?i)(\["\[")''','["',str(x))).apply(lambda x: re.sub(r'''(?i)("]"])''','"]',str(x))).apply(lambda x: re.sub(r', "',',"',str(x)))
heights['height_final']=heights['height_final'].apply(lambda x: re.sub(r'''(?i)(\["\[\\")''','["',str(x)))
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
heights['height_final']=heights['height_final'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
heights['height_final']=heights['height_final'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('(?i)height|\\?"\W?H',' in',str(x))).apply(lambda x: re.sub('(?i)in.?in',' in',str(x)))

del heights['matches_length']
del heights['matches_width']
del heights['matches_diameter']
del heights['matches_ft']
del heights['new_matches']
del heights['blank_matches']

heights['height_final']=heights['height_final'].apply(lambda x: re.sub(r'\.625','63',str(x))).apply(lambda x: re.sub(r'\.188','19',str(x))).apply(lambda x: re.sub(r'12\: in','12 in',str(x))).apply(lambda x: re.sub(r'''\\\\"\\\\xa\s?in''',' in',str(x))).apply(lambda x: re.sub(r'''  ''',' ',str(x)))
regex_pattern_rounding=r'(?i)(\d+\.\d\d\d)|()'         
heights['rounding'] = heights['height_final'].apply(lambda x: re_extract(regex_pattern_rounding, x))
heights[heights['rounding'].astype(str)!='[]']

external_id  \
3581     60906144   
7986     60906359   
23084    60906472   
27075    60906250   
40436    60906397   
43147    60906267   
49245    60906120   
49291    60906113   
59933    60906366   

                                                                       product_name  \
3581                                     Vigo Noma Single Hole Faucet Set in Chrome   
7986                       Vigo Olus Wall Mount Faucet Set in Antique Rubbed Bronze   
23084  Vigo Penela Single Hole Bathroom Faucet with Pop Up in Antique Rubbed Bronze   
27075                                          Viggo Duris Faucet in Brushed Nickel   
40436                        Vigo VG01015MB Satro Single Hole Faucet in Matte Black   
43147                                             Viggo Duris Faucet in Matte Black   
49245                            Vigo Noma Single Hole Faucet in PVD Brushed Nickel   
49291                         Vigo Noma Single Hole Faucet in Antique Rubbed Bronze   
59933                         Vigo Olus Wall Mount Faucet Set in PVD Brushed Nickel   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
3581                                                                                                                                                                                  The Vigo Noma Single Hole Faucet Set brings a sophisticated update to your bathroom. This single lever solid brass faucet for single installations, resists corrosion and tarnishing. Includes a deck plate, all mounting hardware and hot/cold waterlines. Featuring graceful curves, the Noma Faucet Set from Vigo will be a stylish addition to your bathroom Easy single-hole installation Includes solid brass deck plate High-quality ceramic disc cartridge ensures maintenance-free use 1.2 GPM flow rate All mounting hardware and hot/cold waterlines included Corrosion and tarnish-resistant construction Durable solid brass construction Standard US plumbing 3/8\" connections Inserts in a standard 1-3/8\" diameter opening Faucet measures 7.75\" H Spout measures 5\" H Spout reach measures 4.875\" L Deck plate measures 5.5\" L x 2.125\" W x 0.375\" D ADA compliant Watersense certified Limited manufacturer's lifetime warranty Imported Model VG01009CHK1   
7986   Update your shower with Vigo's Olus Wall Mount Faucet Set. This attractive and durable solid brass set includes a single lever wall mount faucet, a mineral resistant nozzle and a drain pop-up. Includes all mounting hardware and hot/cold waterlines. Featuring a stylish and practical design, the Olus Wall Mount Faucet Set from Vigo will be a attractive addition in your bathroom Set includes: Wall mounted single lever for temperature control Wall mounted faucet Pop-up drain plug Easy dual-hole installation High-quality ceramic disc cartridge ensures maintenance-free use 1.2 GPM flow rate All mounting hardware and hot/cold waterlines included Co

In [12]:
heights[curation_col]=heights['height_final']
matches_length=heights[['external_id',curation_col]]
print(len(matches_length))
matches_length

104


external_id                     Q:length
0        69786175  ["17.7 in","18 in","36 in"]
19       69786771    ["75 in","80 in","84 in"]
20       62122306            ["30 in","38 in"]
25       69699019            ["30 in","40 in"]
1072     68623623                 ["14.38 in"]
2511     61900646        ["19.5 in","20.5 in"]
2577     61363236                   ["2.5 in"]
3060     47139053                     ["9 in"]
3581     60906144        ["4.875 in","5.5 in"]
3686     69775596                    ["28 in"]
3828     69736927                    ["20 in"]
4286     69711922                    ["16 in"]
4647     60906427                     ["5 in"]
5247     69786700                    ["64 in"]
5253     69786146    ["74 in","79 in","83 in"]
5907     60386533                  ["13.3 in"]
7571     69786145    ["74 in","79 in","83 in"]
7572     69786157            ["74 in","79 in"]
7986     60906359                 ["7.125 in"]
8336     69597627                  ["9.65 in"]
8565     69780772                    ["27 in"]
8586     69786768    ["75 in","80 in","84 in"]
8588     69775601                    ["16 in"]
8928     69597593                  ["9.65 in"]
9562     69775589                    ["18 in"]
11120    60906342                  ["4.75 in"]
11260    64811284                    ["11 in"]
13369    69765276                  ["4.13 in"]
14225    63577792                   ["5.5 in"]
18095    69749843                 ["13.47 in"]
18097    46695550                    ["18 in"]
18117    69605733    ["75 in","80 in","84 in"]
18118    63191974            ["30 in","38 in"]
18119    62122450            ["30 in","38 in"]
18120    62122313            ["30 in","38 in"]
18121    63334722    ["75 in","80 in","84 in"]
18134    62114356                    ["12 in"]
19448    69698771                 ["12.01 in"]
19738    61910812                    ["28 in"]
21620    60512123            ["93 in","98 in"]
22756    69087233                     ["7 in"]
22913    69786701                    ["64 in"]
23084    60906472                 ["5.125 in"]
23501    69775598                    ["28 in"]
25077    69775602                    ["16 in"]
25078    69775592                    ["18 in"]
25079    69775603                    ["16 in"]
25080    69775599                    ["28 in"]
25122    69786160            ["74 in","79 in"]
25540    69733918                    ["26 in"]
25541    69733917                    ["26 in"]
25648    64857411                  ["15.5 in"]
26629    65160596                  ["3.63 in"]
26630    65160589                  ["3.63 in"]
27075    60906250                 ["4.875 in"]
28662    69786699                    ["64 in"]
28857    63335392    ["75 in","80 in","84 in"]
30514    69786158            ["74 in","79 in"]
31544    69495437                    ["18 in"]
31570    43503032    ["75 in","80 in","84 in"]
31571    69791804                    ["26 in"]
31585    62114387                  ["13.5 in"]
31805    69714424                  ["3.19 in"]
34646    69693280                  ["5.63 in"]
35434    69786142    ["74 in","79 in","83 in"]
35948    47727380                 ["11.25 in"]
36146    62091404                    ["37 in"]
36864    61900660        ["19.5 in","20.5 in"]
37322    45449574                    ["13 in"]
37881    47717992                   ["7.1 in"]
40436    60906397                 ["4.875 in"]
42015    44340476          ["19.5 in","21 in"]
42293    69723253                   ["0.9 in"]
42746    69737255                    ["26 in"]
43147    60906267                 ["4.875 in"]
43524    60961303           ["5 in","9.75 in"]
44532    69786177  ["17.7 in","18 in","36 in"]
44533    69786176  ["17.7 in","18 in","36 in"]
44843    69699034                  ["9.65 in"]
48142    69786698                    ["64 in"]
48143    69786143    ["74 in","79 in","83 in"]
48627    61900653        ["19.5 in","20.5 in"]
49054    60906458                  ["5.25 in"]
49245    60906120                 ["4.875 in"]
49291    60

# Heights

In [13]:
heights=df[(df['matches_height'].astype(str)!='[]')&(df['matches_length'].astype(str)=='[]')]
heights['matches_height']=heights['matches_height'].apply(lambda x: natsorted(x))
## heights['matches_height']=heights['matches_height'].apply(lambda x: fix_text(f'{x}',normalization='NFKC'))
##encoding
# heights['height_final']=heights['matches_height'].apply(lambda x: re.sub(r'''(?i)(\\?\"?\”?(?:inc?h?)?(?<!').(Height|H))|(\\?('')\W?(Height|H))|(\\" H)|(\\ in)|(square\W?Inch)''',' in',str(x))).apply(lambda x: re.sub(r'''(\\?(?:(?<!')'(?!'))?(?:f?t?)?(?:f?o?o?t?)?\W?(Height|H))|(square\W?foot)|(\'\\xa.{0,2}H)''',' ft',str(x))).apply(lambda x: re.sub(r'''(\'.?xa.{0,3}in ft)''',' ft',str(x))).apply(lambda x: re.sub(r'''(?i)(\\ in)''',' in',str(x))).apply(lambda x: re.sub(r'(?i)\"xa0W|xa0W',' in',str(x))).apply(lambda x: re.sub(r'''(?i)\'\\xa in''',' ft',str(x)))
heights['height_final']=heights['matches_height'].apply(lambda x: re.sub(r'''(?i)(\\?\"?\”?(?:inc?h?)?(?<!').(Height|H))|(\\?('')\W?(Height|H))|(\\" H)|(\\ in)|(square\W?Inch)''',' in',str(x))).apply(lambda x: re.sub(r'''(\\?(?:(?<!')'(?!'))?(?:f?t?)?(?:f?o?o?t?)?\W?(Height|H))|(square\W?foot)|(\'\\xa.{0,2}H)''',' ft',str(x))).apply(lambda x: re.sub(r'''(?i)(\\ in)''',' in',str(x)))#.apply(lambda x: re.sub(r'''(\'.?xa.{0,3}in ft)''',' ft',str(x)))#.apply(lambda x: re.sub(r'(?i)\"xa0W|xa0W',' in',str(x)))#.apply(lambda x: re.sub(r'''(?i)\'\\xa in''',' ft',str(x)))
heights['height_final']=heights['height_final'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
heights['height_final']=heights['height_final'].apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'" ','"',str(x))).apply(lambda x: re.sub(r'""','"',str(x))).apply(lambda x: re.sub(r'''(?i)(\\")''','"',str(x)))
heights['height_final']=heights['height_final'].apply(lambda x: re.sub(r'''(?i)(\["\[")''','["',str(x))).apply(lambda x: re.sub(r'''(?i)("]"])''','"]',str(x))).apply(lambda x: re.sub(r', "',',"',str(x)))
heights['height_final']=heights['height_final'].apply(lambda x: re.sub(r'''(?i)(\["\[\\")''','["',str(x)))
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
heights['height_final']=heights['height_final'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
heights['height_final']=heights['height_final'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('(?i)height|\\?"\W?H',' in',str(x))).apply(lambda x: re.sub('(?i)in.?in',' in',str(x)))

del heights['matches_length']
del heights['matches_width']
del heights['matches_diameter']
del heights['matches_ft']
del heights['new_matches']
del heights['blank_matches']

heights['height_final']=heights['height_final'].apply(lambda x: re.sub(r'\.625','63',str(x))).apply(lambda x: re.sub(r'\.188','19',str(x)))
regex_pattern_rounding=r'(?i)(\d+\.\d\d\d)|()'         
heights['rounding'] = heights['height_final'].apply(lambda x: re_extract(regex_pattern_rounding, x))
heights[heights['rounding'].astype(str)!='[]']

Empty DataFrame
Columns: [external_id, product_name, long_desc, buckets, matches_height, blank_matches_long, height_final, rounding]
Index: []

In [14]:
heights['height_final']=heights['height_final'].apply(lambda x: re.sub(r'\["47\.2 in"\,"47\.20 in"\]','["47.2 in"]',str(x)))
heights['height_final'].explode().value_counts()

["8 in"]       3
["62.2 in"]    1
["32 in"]      1
["14.5 in"]    1
Name: height_final, dtype: int64

In [15]:
heights[curation_col]=heights['height_final']
matchesheight=heights[['external_id',curation_col]]
print(len(matchesheight))
matchesheight

6


external_id     Q:length
3616     14241493  ["14.5 in"]
18235    69777594     ["8 in"]
36366    69777596     ["8 in"]
55651    69777595     ["8 in"]
70519    69709858  ["62.2 in"]
71362    69689628    ["32 in"]

In [16]:
# print(len(df[df['matches_diameter'].astype(str)!='[]']))
# height_new[height_new['matches_height'].astype(str)!='[]']

# Length-Good Stuff

In [17]:
from natsort import natsorted
regex_pattern=r'(?i)((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})(?<!x\s)(?<!\d)\d+\.\d+[\W]{0,2}"(?!\sx)[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})(?<!x\s)(?<!\d)\d+.?\d+\/\d+[\W]{0,2}"(?!\sx)[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})(?<!x\s)(?<!\d)(?<!\.)(?<!\/)\d+(?!\.\d)[\W]{0,2}"(?!\sx)[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})(?<!x\s)(?<!\d)(?<!\.)(?<!\/)\d+(?!\.)[\W]{0,2}"(?!\sx)[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|()'           
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[df['matches'].astype(str)!='[]']))
df['matches']=df['matches'].apply(lambda x: natsorted(x))

75


In [18]:
# regex_pattern_blank=r'\d'
# df['new_matches'] = df['matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
# df_new=df[(df['new_matches'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')]
# df_new['matches_length']=df_new['matches'].apply(lambda x: re.sub(r'(?i)\\" L|"L|L|"|length',' in',str(x))).apply(lambda x: re.sub(r'\\" L|"L|\"xa0L|xa0L|"|xa0','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))

# df_new['matches_lengths']=df_new['matches_length'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# df_new['matches_lengths']=df_new['matches_lengths'].apply(lambda x: re.sub('\\\" L|"',' in',str(x))).apply(lambda x: re.sub(r'(?i)(foot)|(feet)','ft',str(x))).apply(lambda x: re.sub(r'(?i)(inch)|(in)','in',str(x))).apply(lambda x: re.sub('\\\\','',str(x))).apply(lambda x: re.sub(',','","',str(x))).apply(lambda x: re.sub('" | "','"',str(x)))
# import ast
# def remove_duplicates(A):
#     [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
#     return A
# df_new['matches_lengths']=df_new['matches_lengths'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
# df_new['matches_lengths']=df_new['matches_lengths'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('\”|\)|inig','',str(x))).apply(lambda x: re.sub('  ',' ',str(x)))

# del df_new['matches_height']
# del df_new['matches_ft']
# del df_new['new_matches']
# del df_new['blank_matches']
# del df_new['blank_matches_long']
# # del df_new['matches']

# df_new['matches_lengths']=df_new['matches_lengths'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
# df_new['matches_lengths']=df_new['matches_lengths'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
# df_new['matches_lengths']=df_new['matches_lengths'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))

In [19]:
regex_pattern_blank=r'\d'
df['new_matches'] = df['matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df_new=df[(df['new_matches'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')]
df_new['matches_length']=df_new['matches'].apply(lambda x: re.sub(r'''(?i)[^\d"”]{0,3}((?<!')'(?!'))[^\d"”]{0,4}(Length|L)|(\D{0,3}(foot|ft|feet))''',' ft',str(x))).apply(lambda x: re.sub(r'''(?i)[^\d']{0,3}(["”]).{0,4}(Length|L)|(\D{0,3}('')\W?(Length|L))|(\D{0,3}(inc?h?))''',' in',str(x)))                      
df_new['matches_length']=df_new['matches_length'].apply(lambda x: re.sub(r'(\\?\\?u200 in)','in',str(x))).apply(lambda x: re.sub(r'(\\?\\?u200 ft)','ft',str(x)))

df_new['matches_lengths']=df_new['matches_length'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
df_new['matches_lengths']=df_new['matches_lengths'].apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'" ','"',str(x))).apply(lambda x: re.sub(r'""','"',str(x))).apply(lambda x: re.sub(r'''(?i)(\\")''','"',str(x)))
df_new['matches_lengths']=df_new['matches_lengths'].apply(lambda x: re.sub(r'''(?i)(\["\[")''','["',str(x))).apply(lambda x: re.sub(r'''(?i)("]"])''','"]',str(x))).apply(lambda x: re.sub(r', "',',"',str(x)))
df_new['matches_lengths']=df_new['matches_lengths'].apply(lambda x: re.sub(r'''(?i)(\["\[\\")''','["',str(x)))
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

del df_new['matches_height']
del df_new['matches_ft']
del df_new['new_matches']
del df_new['blank_matches']
del df_new['blank_matches_long']
# del df_new['matches_height']
del df_new['matches_width']
del df_new['matches_diameter']
# del df_new['matches_ft']
# del df_new['new_matches']
# del df_new['blank_matches']
# del df_new['blank_matches_long']

# del df_new['matches']
df_new['matches_lengths']=df_new['matches_lengths'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
df_new['matches_lengths']=df_new['matches_lengths'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('(?i)\.1245?','.12',str(x))).apply(lambda x: re.sub('(?i)(?<!\s)in',' in',str(x)))


regex_pattern_rounding=r'(?i)(\d+\.\d\d\d)|(\.\d+0)|()'         
df_new['rounding'] = df_new['matches_lengths'].apply(lambda x: re_extract(regex_pattern_rounding, x))
df_new[df_new['rounding'].astype(str)!='[]']

external_id  \
3581     60906144   
7986     60906359   
23084    60906472   
27075    60906250   
40436    60906397   
43147    60906267   
49245    60906120   
49291    60906113   
59933    60906366   

                                                                       product_name  \
3581                                     Vigo Noma Single Hole Faucet Set in Chrome   
7986                       Vigo Olus Wall Mount Faucet Set in Antique Rubbed Bronze   
23084  Vigo Penela Single Hole Bathroom Faucet with Pop Up in Antique Rubbed Bronze   
27075                                          Viggo Duris Faucet in Brushed Nickel   
40436                        Vigo VG01015MB Satro Single Hole Faucet in Matte Black   
43147                                             Viggo Duris Faucet in Matte Black   
49245                            Vigo Noma Single Hole Faucet in PVD Brushed Nickel   
49291                         Vigo Noma Single Hole Faucet in Antique Rubbed Bronze   
59933                         Vigo Olus Wall Mount Faucet Set in PVD Brushed Nickel   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
3581                                                                                                                                                                                  The Vigo Noma Single Hole Faucet Set brings a sophisticated update to your bathroom. This single lever solid brass faucet for single installations, resists corrosion and tarnishing. Includes a deck plate, all mounting hardware and hot/cold waterlines. Featuring graceful curves, the Noma Faucet Set from Vigo will be a stylish addition to your bathroom Easy single-hole installation Includes solid brass deck plate High-quality ceramic disc cartridge ensures maintenance-free use 1.2 GPM flow rate All mounting hardware and hot/cold waterlines included Corrosion and tarnish-resistant construction Durable solid brass construction Standard US plumbing 3/8\" connections Inserts in a standard 1-3/8\" diameter opening Faucet measures 7.75\" H Spout measures 5\" H Spout reach measures 4.875\" L Deck plate measures 5.5\" L x 2.125\" W x 0.375\" D ADA compliant Watersense certified Limited manufacturer's lifetime warranty Imported Model VG01009CHK1   
7986   Update your shower with Vigo's Olus Wall Mount Faucet Set. This attractive and durable solid brass set includes a single lever wall mount faucet, a mineral resistant nozzle and a drain pop-up. Includes all mounting hardware and hot/cold waterlines. Featuring a stylish and practical design, the Olus Wall Mount Faucet Set from Vigo will be a attractive addition in your bathroom Set includes: Wall mounted single lever for temperature control Wall mounted faucet Pop-up drain plug Easy dual-hole installation High-quality ceramic disc cartridge ensures maintenance-free use 1.2 GPM flow rate All mounting hardware and hot/cold waterlines included Co

In [20]:
print(len(df_new))
df_new[curation_col]=df_new['matches_lengths']
matchesnew=df_new[['external_id',curation_col]]
matchesnew[curation_col]=matchesnew[curation_col].apply(lambda x: re.sub(r'\.00','',str(x))).apply(lambda x: re.sub(r'\.10','.1',str(x))).apply(lambda x: re.sub(r'\.20','.2',str(x))).apply(lambda x: re.sub(r'\.30','.3',str(x))).apply(lambda x: re.sub(r'\.40','.4',str(x))).apply(lambda x: re.sub(r'\.50','.5',str(x))).apply(lambda x: re.sub(r'\.60','.6',str(x))).apply(lambda x: re.sub(r'\.70','.7',str(x))).apply(lambda x: re.sub(r'\.80','.8',str(x))).apply(lambda x: re.sub(r'\.90','.9',str(x)))
# matchesnew=matchesnew[matchesnew[curation_col]!='["17 in: inength","20 in; inength","23 in; inength","26 in; inength","29 in; inength"]']
matchesnew[curation_col].explode().value_counts()
matchesnew[0:500]

75


external_id                     Q:length
0        69786175  ["17.7 in","18 in","36 in"]
1072     68623623                 ["14.38 in"]
2511     61900646        ["19.5 in","20.5 in"]
2577     61363236                   ["2.5 in"]
3060     47139053                     ["9 in"]
3581     60906144        ["4.875 in","5.5 in"]
3686     69775596                    ["28 in"]
4286     69711922                    ["16 in"]
4647     60906427                     ["5 in"]
5253     69786146    ["74 in","79 in","83 in"]
5907     60386533                  ["13.3 in"]
7571     69786145    ["74 in","79 in","83 in"]
7572     69786157            ["74 in","79 in"]
7986     60906359                 ["7.125 in"]
8336     69597627                  ["9.65 in"]
8565     69780772                    ["27 in"]
8588     69775601                    ["16 in"]
8928     69597593                  ["9.65 in"]
11120    60906342                  ["4.75 in"]
13369    69765276                  ["4.13 in"]
18095    69749843                 ["13.47 in"]
18117    69605733    ["75 in","80 in","84 in"]
18134    62114356                    ["12 in"]
19448    69698771                 ["12.01 in"]
19738    61910812                    ["28 in"]
22756    69087233                     ["7 in"]
23084    60906472                 ["5.125 in"]
23501    69775598                    ["28 in"]
25077    69775602                    ["16 in"]
25079    69775603                    ["16 in"]
25080    69775599                    ["28 in"]
25122    69786160            ["74 in","79 in"]
25648    64857411                  ["15.5 in"]
26629    65160596                  ["3.63 in"]
26630    65160589                  ["3.63 in"]
27075    60906250                 ["4.875 in"]
30514    69786158            ["74 in","79 in"]
31585    62114387                  ["13.5 in"]
31805    69714424                  ["3.19 in"]
34646    69693280                  ["5.63 in"]
35434    69786142    ["74 in","79 in","83 in"]
35948    47727380                 ["11.25 in"]
36146    62091404                    ["37 in"]
36864    61900660        ["19.5 in","20.5 in"]
37322    45449574                    ["13 in"]
37881    47717992                   ["7.1 in"]
40436    60906397                 ["4.875 in"]
42015    44340476          ["19.5 in","21 in"]
42293    69723253                   ["0.9 in"]
43147    60906267                 ["4.875 in"]
43524    60961303                     ["5 in"]
44532    69786177  ["17.7 in","18 in","36 in"]
44533    69786176  ["17.7 in","18 in","36 in"]
44843    69699034                  ["9.65 in"]
48143    69786143    ["74 in","79 in","83 in"]
48627    61900653        ["19.5 in","20.5 in"]
49054    60906458                  ["5.25 in"]
49245    60906120                 ["4.875 in"]
49291    60906113                 ["4.875 in"]
49404    69775597                    ["28 in"]
55129    69775600                    ["26 in"]
59482    69711923                    ["16 in"]
59933    60906366                 ["7.125 in"]
60813    60906465                  ["5.25 in"]
61481    69714419                  ["3.19 in"]
61923    60906410                     ["5 in"]
63197    60906328                   ["6.5 in"]
64392    65160602                  ["3.63 in"]
65559    60906441                  ["5.25 in"]
66657    69786159            ["74 in","79 in"]
66672    69698770                 ["12.01 in"]
69226    61911000                    ["30 in"]
69589    69597628                  ["11.8 in"]
70522    69699035                  ["9.65 in"]
70603    60906335                  ["4.75 in"]

# NA Values

In [21]:
regex_pattern_na=r'''(?i)((?<!Cord Measures.{10})\d+\.\d+.{0,2}["'].{0,2}[LWHD]?\b)|((?<!Cord Measures.{10})\d+.?\d+\/\d+.{0,2}["'].{0,2}[LWHD]?\b)|((?<!Cord Measures.{10})(?<!\.)(?<!\/)\d+(?!\.\d).{0,2}["'].{0,2}[LWHD]?\b)|((?<!Cord Measures.{10})(?<!\.)(?<!\/)\d+(?!\.).{0,2}["'].{0,2}[LWHD]?\b)|((?<!\.)\d+(?!\.\d).{0,3}?diam)|(measure)|()'''
regex_pattern_measure=r'(?i)(measure)|()'
df_na=df
df_na['na_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))
df_na['na_measure_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_measure, x))
print(len(df_na[(df_na['na_matches'].astype(str)=='[]')&(df_na['na_measure_matches'].astype(str)=='[]')&(df_na['blank_matches'].astype(str)=='[]')]))
df_na=df_na[(df_na['na_matches'].astype(str)=='[]')&(df_na['na_measure_matches'].astype(str)=='[]')&(df_na['blank_matches'].astype(str)=='[]')]
df_na[curation_col]='n/a'
matchesna=df_na[['external_id',curation_col]]
print(len(matchesna))
matchesna

5
5


external_id Q:length
26009    69723347      n/a
28057    69723344      n/a
33643    46131362      n/a
47793    69723345      n/a
59400    69723346      n/a

# Max Min (Length is the max)

In [22]:
pat_split=r'''(?i)(\d+.?\d*\/?\d*\W?(?:(?:I?n?c?h?"?)?(?:f?e?e?t?'?)?(?:f?o?o?t?'?)?)\W?x.?\d+\W?\d*\/?\d*\W?(?:(?:I?n?c?h?"?)?(?:f?e?e?t?'?)?(?:f?o?o?t?'?)?)\W?)|()'''
df['matches_split'] = df['product_name'].apply(lambda x: re_extract(pat_split, str(x)))
regex_pattern_blank=r'(?i)(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
df['matches'] = df['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
split=df[(df['matches_split'].astype(str)!='[]')&(df['matches_length'].astype(str)=='[]')]
print(len(split))
# split.head()

4


In [23]:
split['matches_split']=split['matches_split'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))])
split['matches_split']=split['matches_split'].str.get(0).apply(lambda x: x.lower())
split[['length', 'width']]=split['matches_split'].str.split("x",expand=True,)

In [24]:
# split[split['width'].astype(str)=="24"]

In [25]:
split['length']=split['length'].apply(lambda x: re.sub(r"\[\'",'',str(x))).apply(lambda x: re.sub(r"(?i)\W?feet|\W?foot|\\?\\?\'",'ft',str(x))).apply(lambda x: re.sub(r"ft\s?ft",'ft',str(x))).apply(lambda x: re.sub(r'-inch|-inc|\\?\\?\"',' in',str(x))).apply(lambda x: re.sub(r"\['|']|8x/1x|4x4 b|26x31 g|8x/1",'',str(x))).apply(lambda x: re.sub(r"ft",' ft',str(x))).apply(lambda x: re.sub(r"2 24 in",'24 in',str(x))).apply(lambda x: re.sub(r"33 (?!in)",'33 in',str(x))).apply(lambda x: re.sub(r" ",'',str(x))).apply(lambda x: re.sub(r"in",' in',str(x))).apply(lambda x: re.sub(r"5 inch",'31.5 in',str(x))).apply(lambda x: re.sub(r"20x24','20",'20 in',str(x)))                        
split['width']=split['width'].apply(lambda x: re.sub(r"\'\]",'',str(x))).apply(lambda x: re.sub(r"(?i)\W?feet|\W?foot|\\?\\?\'",'ft',str(x))).apply(lambda x: re.sub(r"ft\s?ft",'ft',str(x))).apply(lambda x: re.sub(r'-inch|-inc|\\?\\?\"',' in',str(x))).apply(lambda x: re.sub(r"\['|']|None| |4-f",'',str(x))).apply(lambda x: re.sub(r"in",' in',str(x))).apply(lambda x: re.sub(r"ft",' ft',str(x))).apply(lambda x: re.sub(r"24 in','20x24 in",'24 in',str(x))).apply(lambda x: re.sub(r"27b",'27 in',str(x))).apply(lambda x: re.sub(r"24(?! )",'24 in',str(x)))
print(len(split['width'].explode().value_counts()))
# split['width'].explode().value_counts()

4


In [26]:
del split['buckets']
del split['matches_width']
del split['matches_diameter']
del split['matches_ft']
del split['new_matches']
del split['na_measure_matches']

In [27]:
split=split[split['length'].astype(str)!=""]
split=split[split['width'].astype(str)!=""]

split_inch=split
pat_split=r'''(?i)(in)|()'''
split_inch['inch'] = split_inch['width'].apply(lambda x: re_extract(pat_split, str(x)))
split_inch=split_inch[split_inch['inch'].astype(str)!='[]']
split_inch['length']=split_inch['length'].apply(lambda x:re.sub(r'\s?in|\s?inch|ch','',str(x)))
split_inch['width']=split_inch['width'].apply(lambda x:re.sub(r'\s?in|\s?inch|ch','',str(x))).apply(lambda x: re.sub(r'','',str(x)))
split_inch['length']=split_inch['length'].astype(float)
split_inch['width']=split_inch['width'].astype(float)
split_inch['Max'] = split_inch[['length', 'width']].values.max(1)
split_inch['Min'] = split_inch[['length', 'width']].values.min(1)
split_inch['Max']=split_inch['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
print(len(split_inch))
split_inch[curation_col]=split_inch['Max']
split_inch[curation_col]=split_inch[curation_col].apply(lambda x: str(f"['{x} in']")).apply(lambda x: re.sub(r"'",'"',str(x)))


split_ft=split
pat_split=r'''(?i)(ft)|()'''
split_ft['inch'] = split_ft['length'].apply(lambda x: re_extract(pat_split, str(x)))
pat_split=r'''(?i)(ft)|()'''
split_ft['ft'] = split_ft['width'].apply(lambda x: re_extract(pat_split, str(x)))
split_ft=split_ft[(split_ft['ft'].astype(str)!='[]')&(split_ft['inch'].astype(str)!='[]')]
split_ft['length']=split_ft['length'].apply(lambda x:re.sub(r'\W?ft','',str(x)))
split_ft['width']=split_ft['width'].apply(lambda x:re.sub(r'\W?ft','',str(x))).apply(lambda x: re.sub(r'','',str(x)))
split_ft['length']=split_ft['length'].astype(float)
split_ft['width']=split_ft['width'].astype(float)
split_ft['Max'] = split_ft[['length', 'width']].values.max(1)
split_ft['Min'] = split_ft[['length', 'width']].values.min(1)
split_ft['Max']=split_ft['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
print(len(split_ft))

split_ft[curation_col]=split_ft['Max']
split_ft[curation_col]=split_ft[curation_col].apply(lambda x: str(f"['{x} ft']")).apply(lambda x: re.sub(r"'",'"',str(x)))

1
0


In [28]:
msplit_inch=split_inch[['external_id',curation_col]]
msplit_ft=split_ft[['external_id',curation_col]]
msplit_inch

external_id   Q:length
71362    69689628  ["32 in"]

In [29]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name
print(get_df_name(matchesna))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name
print(get_df_name(msplit_inch))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(msplit_ft))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesnew))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesheight))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesheight))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matches_length))

matchesna
msplit_inch
msplit_ft
matchesnew
matchesheight
matchesheight
matches_length


In [31]:
print(len(df))
print(len(matchesna)+len(msplit_inch)+len(msplit_ft)+len(matchesnew)+len(matches_length))

501
185


In [30]:
stop   

NameError: name 'stop' is not defined

# send to the folder for upload

In [32]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute, df, matches_length)

In [33]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute, df, matchesheight)    

In [34]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('Bed Bath & Beyond', attribute, df, matchesna)

In [35]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False)

looks_good('Bed Bath & Beyond', attribute, df, msplit_inch)

In [36]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False)

looks_good('Bed Bath & Beyond', attribute, df, msplit_ft)

In [37]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False)

looks_good('Bed Bath & Beyond', attribute, df, matchesnew)